##Importación de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Instalar las librerias necesarias para la ejecución

!pip install gdown

import gdown
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# ID del archivo en Google Drive
file_id = "1nWl7vNpI2XHd_B6cwAgIAYClQ3J9F6r1"
download_url = f"https://drive.google.com/uc?id={file_id}"

# Descargar el archivo CSV a local
output = "archivo.csv"
gdown.download(download_url, output, quiet=False)

# Leer CSV con pandas
df = pd.read_csv(output)



# Importacion del test

file_id = "1c9lxChxn6YhWD9YJCBSdxmA3Xt-fasN7"
download_url = f"https://drive.google.com/uc?id={file_id}"

output = "archivo.csv"
gdown.download(download_url, output, quiet=False)

df_test=pd.read_csv(output)


El dataframe lo guardamos en la variable df y el que vamos a modificar va a estar en la variable dfa

In [ ]:
df.head()

##Lo primero que vamos a hacer es resolver el problema de los datos vacios

In [ ]:
print("El porcentaje de datos vacios en el df es de: ")
print((((df.isna().sum().sum())/(df.shape[0]*df.shape[1]))*(100)),"%")


Vemos que tenemos demasiados valores vacios, estos representan aproximadamente el 2% del data frame

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import unicodedata
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def preprocess_dataframe(df_input):
    """
    Preprocesa el DataFrame completamente:
    - Convierte TODO a numérico
    - Elimina columnas duplicadas
    - Maneja valores faltantes
    - Optimiza tipos de datos
    """
    df = df_input.copy()
    df = df.replace(['nan', 'NaN', '', 'None', 'null'], np.nan)
    cols_to_drop = ['ID', 'PERIODO_ACADEMICO']
    existing_to_drop = [col for col in cols_to_drop if col in df.columns]
    if existing_to_drop:
        df = df.drop(columns=existing_to_drop)
        print("Eliminadas columnas: {existing_to_drop}")

    target_col = 'RENDIMIENTO_GLOBAL'
    has_target = target_col in df.columns
    cols_one_hot = ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO']
    cols_ordinales = [
        'E_VALORMATRICULAUNIVERSIDAD',
        'E_HORASSEMANATRABAJA',
        'F_ESTRATOVIVIENDA',
        'F_EDUCACIONPADRE',
        'F_EDUCACIONMADRE'
    ]

    # Columnas binarias
    cols_binarias = [
        'F_TIENEINTERNET',
        'F_TIENELAVADORA',
        'F_TIENEAUTOMOVIL',
        'E_PRIVADO_LIBERTAD',
        'E_PAGOMATRICULAPROPIO',
        'F_TIENECOMPUTADOR',
        'F_TIENEINTERNET_1'
    ]

    # Columnas numéricas
    cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

    # 4. ONE-HOT ENCODING
    cols_to_encode = [col for col in cols_one_hot if col in df.columns]

    if cols_to_encode:
        print("Aplicando One-Hot Encoding a: {cols_to_encode}")
        for col in cols_to_encode:
            # Rellenar NaN con 'Desconocido'
            df[col] = df[col].fillna('Desconocido').astype(str)

        # One-hot encoding
        df = pd.get_dummies(df, columns=cols_to_encode, prefix=cols_to_encode,
                           dtype='int8', drop_first=False)

    # 5. ORDINAL ENCODING

    # 5.1 E_VALORMATRICULAUNIVERSIDAD
    if 'E_VALORMATRICULAUNIVERSIDAD' in df.columns:
        print("Procesando E_VALORMATRICULAUNIVERSIDAD")
        orden_matricula = {
            'No pagó matrícula': 0,
            'Menos de 500 mil': 1,
            'Entre 500 mil y menos de 1 millón': 2,
            'Entre 1 millón y menos de 2.5 millones': 3,
            'Entre 2.5 millones y menos de 4 millones': 4,
            'Entre 4 millones y menos de 5.5 millones': 5,
            'Entre 5.5 millones y menos de 7 millones': 6,
            'Más de 7 millones': 7
        }
        df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].astype(str).map(orden_matricula)
        df['E_VALORMATRICULAUNIVERSIDAD'] = pd.to_numeric(df['E_VALORMATRICULAUNIVERSIDAD'], errors='coerce')
        moda = df['E_VALORMATRICULAUNIVERSIDAD'].mode()[0] if len(df['E_VALORMATRICULAUNIVERSIDAD'].mode()) > 0 else 3
        df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].fillna(moda).astype('int8')

    # 5.2 E_HORASSEMANATRABAJA
    if 'E_HORASSEMANATRABAJA' in df.columns:
        print("Procesando E_HORASSEMANATRABAJA")
        orden_horas = {
            '0': 0,
            'Menos de 10 horas': 1,
            'Entre 11 y 20 horas': 2,
            'Entre 21 y 30 horas': 3,
            'Más de 30 horas': 4
        }
        df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].astype(str).map(orden_horas)
        df['E_HORASSEMANATRABAJA'] = pd.to_numeric(df['E_HORASSEMANATRABAJA'], errors='coerce')
        moda = df['E_HORASSEMANATRABAJA'].mode()[0] if len(df['E_HORASSEMANATRABAJA'].mode()) > 0 else 1
        df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].fillna(moda).astype('int8')

    # 5.3 F_ESTRATOVIVIENDA
    if 'F_ESTRATOVIVIENDA' in df.columns:
        orden_estrato = {
            'Sin Estrato': 0,
            'Estrato 1': 1,
            'Estrato 2': 2,
            'Estrato 3': 3,
            'Estrato 4': 4,
            'Estrato 5': 5,
            'Estrato 6': 6
        }
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].astype(str).map(orden_estrato)
        df['F_ESTRATOVIVIENDA'] = pd.to_numeric(df['F_ESTRATOVIVIENDA'], errors='coerce')
        mediana = df['F_ESTRATOVIVIENDA'].median()
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].fillna(mediana).astype('int8')

    # 5.4 F_EDUCACIONPADRE
    if 'F_EDUCACIONPADRE' in df.columns:
        orden_educacion = {
            'Ninguno': 0,
            'Primaria incompleta': 1,
            'Primaria completa': 2,
            'Secundaria (Bachillerato) incompleta': 3,
            'Secundaria (Bachillerato) completa': 4,
            'Técnica o tecnológica incompleta': 5,
            'Técnica o tecnológica completa': 6,
            'Educación profesional incompleta': 7,
            'Educación profesional completa': 8,
            'Postgrado': 9,
            'No sabe': 0,
            'No Aplica': 0
        }
        df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].astype(str).map(orden_educacion)
        df['F_EDUCACIONPADRE'] = pd.to_numeric(df['F_EDUCACIONPADRE'], errors='coerce')
        mediana = df['F_EDUCACIONPADRE'].median()
        df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].fillna(mediana).astype('int8')

    # 5.5 F_EDUCACIONMADRE
    if 'F_EDUCACIONMADRE' in df.columns:
        orden_educacion = {
            'Ninguno': 0,
            'Primaria incompleta': 1,
            'Primaria completa': 2,
            'Secundaria (Bachillerato) incompleta': 3,
            'Secundaria (Bachillerato) completa': 4,
            'Técnica o tecnológica incompleta': 5,
            'Técnica o tecnológica completa': 6,
            'Educación profesional incompleta': 7,
            'Educación profesional completa': 8,
            'Postgrado': 9,
            'No Aplica': 0
        }
        df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].astype(str).map(orden_educacion)
        df['F_EDUCACIONMADRE'] = pd.to_numeric(df['F_EDUCACIONMADRE'], errors='coerce')
        mediana = df['F_EDUCACIONMADRE'].median()
        df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].fillna(mediana).astype('int8')

    # 6. COLUMNAS BINARIAS (Si/No, S/N)
    for col in cols_binarias:
        if col in df.columns:
            # Convertir todo a string primero
            df[col] = df[col].astype(str).str.strip().str.lower()

            # Mapear a 0/1
            mapeo = {
                'si': 1, 'sí': 1, 's': 1, '1': 1, '1.0': 1, 'true': 1,
                'no': 0, 'n': 0, '0': 0, '0.0': 0, 'false': 0,
                'nan': np.nan, 'none': np.nan
            }
            df[col] = df[col].map(mapeo)
            df[col] = pd.to_numeric(df[col], errors='coerce')

            # Rellenar NaN con moda
            moda = df[col].mode()[0] if len(df[col].mode()) > 0 else 0
            df[col] = df[col].fillna(moda).astype('int8')

    # 7. VARIABLE OBJETIVO (RENDIMIENTO_GLOBAL)
    if has_target:
        orden_rendimiento = {
            'bajo': 0,
            'medio-bajo': 1,
            'medio-alto': 2,
            'alto': 3
        }
        df[target_col] = df[target_col].astype(str).str.lower().str.strip().map(orden_rendimiento)
        df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
        # NO rellenar NaN en el target, dejar como está
        df[target_col] = df[target_col].astype('int8')

    # 8. COLUMNAS NUMÉRICAS (INDICADORES)
    for col in cols_numericas:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            mediana = df[col].median()
            df[col] = df[col].fillna(mediana).astype('float32')

    # 9. PROCESAR CUALQUIER COLUMNA RESTANTE (por si acaso)
    for col in df.columns:
        if df[col].dtype == 'object':
            # Intentar convertir a numérico
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # Rellenar NaN con 0
            df[col] = df[col].fillna(0).astype('float32')

    # 10. LIMPIAR NOMBRES DE COLUMNAS (quitar caracteres especiales)

    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
    df.columns = df.columns.str.strip('_')
    df.columns = df.columns.str.replace('__+', '_', regex=True)  # Reemplazar múltiples _ por uno solo

    # 11. ELIMINAR COLUMNAS DUPLICADAS
    duplicadas_antes = df.columns.duplicated().sum()
    if duplicadas_antes > 0:
        df = df.loc[:, ~df.columns.duplicated()]



    # Verificar que todo es numérico
    non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()

    # Verificar NaN
    nan_counts = df.isnull().sum()
    if nan_counts.sum() > 0:
        print(f"\n Valores NaN encontrados en:")
        print(nan_counts[nan_counts > 0])
    else:
        print(" No hay valores NaN")

    print("="*60 + "\n")

    return df

In [ ]:
def alinear_columnas(X_train, X_test):
    """Alinea columnas de X_test con X_train"""
    # Agregar columnas faltantes con 0
    missing_cols = set(X_train.columns) - set(X_test.columns)
    for col in missing_cols:
        X_test[col] = 0

    # Eliminar columnas extra
    extra_cols = set(X_test.columns) - set(X_train.columns)
    X_test = X_test.drop(columns=list(extra_cols), errors='ignore')

    # Reordenar para que coincidan
    X_test = X_test[X_train.columns]

    return X_test


In [ ]:
df.dtypes.unique()

#A partir de este momento queda listo el preprocesado, entonces vamos a comenzar con el primer modelo (Random Forest)

In [ ]:
# Usar la función para preprocesar el DataFrame original
df = preprocess_dataframe(df)


In [ ]:
df.dtypes.unique()

### Uso de la función de preprocesamiento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
!pip install lightgbm
import lightgbm as lgb


##Hacemos los splits y la distribucion de los datos X y Y

In [ ]:
from sklearn.model_selection import train_test_split

TARGET = "RENDIMIENTO_GLOBAL"

y_train = df[TARGET]
X_train = df.drop(columns=[TARGET])

X_test = preprocess_dataframe(df_test)
X_test = alinear_columnas(X_train, X_test)





In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# 1. ENTRENAR
print("Entrenando Random Forest...")
model_rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

model_rf.fit(X_train, y_train)
print("Entrenamiento completado!")

# 2. PREDECIR EN TEST
print("Realizando predicciones en test...")
y_test_pred = model_rf.predict(X_test)

# 3. CREAR DATAFRAME CON RESULTADOS
class_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}

resultados = pd.DataFrame({
    'ID': range(len(X_test)),  # O usa X_test.index si tiene IDs
    'RENDIMIENTO_GLOBAL': [class_mapping[p] for p in y_test_pred]
})

# 4. MOSTRAR Y GUARDAR
print("Distribución de predicciones:")
print(resultados['RENDIMIENTO_GLOBAL'].value_counts())
print("Primeras predicciones:")
print(resultados.head())

resultados.to_csv('predicciones_random_forest.csv', index=False)
print("Predicciones guardadas!")




In [ ]:
# PREDECIR EN TEST
print("Realizando predicciones en test...")
y_test_pred = model_rf.predict(X_test)

# Crear DataFrame con ID y predicción
class_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}

resultados = pd.DataFrame({
    'ID': df_test["ID"],  # Usa el índice de X_test como ID
    'RENDIMIENTO_GLOBAL': [class_mapping[p] for p in y_test_pred]
})

print("Distribución de predicciones en test:")
print(resultados['RENDIMIENTO_GLOBAL'].value_counts())
print("Primeras 5 predicciones:")
print(resultados.head())

# Guardar predicciones
resultados.to_csv('predicciones_random_forest.csv', index=False)
print("Predicciones guardadas en 'predicciones_random_forest.csv'")

In [ ]:
resultados.head()